In [13]:
import math
import random
import time

def is_common_password(password, common_passwords):
    return password in common_passwords

def main(password):
        clear_screen()


        # Salt generation
        salt = generate_salt()

        # Hash the password with salt
        hashed_password = hash_password(password, salt)

        # Print salt and hashed password (you should store them securely)
        print("Salt:", salt)
        print("Hashed Password:", hashed_password)

        # Your password strength checking code goes here
        # ...


        excess = len(password) - 6
        base_score = 50

        num_upper = 0
        num_symbol = 0
        num_lower = 0
        num_digit = 0

        bonus_excess = 3
        bonus_upper = 4
        bonus_numbers = 5
        bonus_symbols = 5
        bonus_combo = 0
        bonus_flat_lower = 0
        bonus_flat_number = 0

        for ch in password:
            if '0' <= ch <= '9':
                num_digit += 1
            if 'a' <= ch <= 'z':
                num_lower += 1
            if 'A' <= ch <= 'Z':
                num_upper += 1
            if '!' <= ch <= '/':
                num_symbol += 1

        if num_upper > 0 and num_digit > 0 and num_symbol > 0:
            bonus_combo = 25
        elif (num_upper > 0 and num_digit > 0) or (num_upper > 0 and num_symbol > 0) or (num_digit > 0 and num_symbol > 0):
            bonus_combo = 15

        score = base_score + (excess * bonus_excess) + (num_upper * bonus_upper) + \
            (num_digit * bonus_numbers) + (num_symbol * bonus_symbols) + \
            bonus_combo + bonus_flat_lower + bonus_flat_number

        print("Checking Password Strength...", end="")
        for _ in range(3):
            print("█", end="")
            time.sleep(0.1)
        print()


        print()
        if score < 50:
            print("Password Status: Weak")
        elif 50 <= score < 75:
            print("Password Status: Average")
        elif 75 <= score < 100:
            print("Password Status: Strong")
        elif score >= 100:
            print("Password Status: Secure")

        # Now, calculate and display entropy
        R = 26 + 21
        entropy = math.log(math.pow(R, len(password))) / math.log(2)
        print("The entropy of this password is", entropy)

        # Recheck password strength based on entropy
        recheck_password_strength(entropy)


def generate_salt():
    # You can use any method to generate a random salt.
    # Here, we're generating a simple salt using random integers.
    salt = ""
    for _ in range(16):  # 16 characters long salt
        salt += chr(random.randint(32, 126))  # Generate random printable ASCII characters
    return salt

def hash_password(password, salt):
    # Combine the password and salt
    salted_password = password + salt

    # Initialize the SHA-256 constants
    h0 = 0x6a09e667
    h1 = 0xbb67ae85
    h2 = 0x3c6ef372
    h3 = 0xa54ff53a
    h4 = 0x510e527f
    h5 = 0x9b05688c
    h6 = 0x1f83d9ab
    h7 = 0x5be0cd19

    # Process the salted password
    for chunk in range(0, len(salted_password), 64):
        w = [0] * 64

        for i in range(16):
            w[i] = int.from_bytes(salted_password[chunk + i].encode(), 'big')

        for i in range(16, 64):
            s0 = (rightrotate(w[i - 15], 7) ^ rightrotate(w[i - 15], 18) ^ (w[i - 15] >> 3))
            s1 = (rightrotate(w[i - 2], 17) ^ rightrotate(w[i - 2], 19) ^ (w[i - 2] >> 10))
            w[i] = (w[i - 16] + s0 + w[i - 7] + s1) & 0xFFFFFFFFFFFFFFFF

        a, b, c, d, e, f, g, h = h0, h1, h2, h3, h4, h5, h6, h7

        for i in range(64):
            S1 = rightrotate(e, 6) ^ rightrotate(e, 11) ^ rightrotate(e, 25)
            ch = (e & f) ^ ((~e) & g)
            temp1 = h + S1 + ch + k[i] + w[i]
            S0 = rightrotate(a, 2) ^ rightrotate(a, 13) ^ rightrotate(a, 22)
            maj = (a & b) ^ (a & c) ^ (b & c)
            temp2 = S0 + maj

            h = g
            g = f
            f = e
            e = (d + temp1) & 0xFFFFFFFFFFFFFFFF
            d = c
            c = b
            b = a
            a = (temp1 + temp2) & 0xFFFFFFFFFFFFFFFF

        h0 = (h0 + a) & 0xFFFFFFFFFFFFFFFF
        h1 = (h1 + b) & 0xFFFFFFFFFFFFFFFF
        h2 = (h2 + c) & 0xFFFFFFFFFFFFFFFF
        h3 = (h3 + d) & 0xFFFFFFFFFFFFFFFF
        h4 = (h4 + e) & 0xFFFFFFFFFFFFFFFF
        h5 = (h5 + f) & 0xFFFFFFFFFFFFFFFF
        h6 = (h6 + g) & 0xFFFFFFFFFFFFFFFF
        h7 = (h7 + h) & 0xFFFFFFFFFFFFFFFF

    # Calculate the final hash
    digest = '{:x}'.format(h0) + '{:x}'.format(h1) + '{:x}'.format(h2) + '{:x}'.format(h3) + \
             '{:x}'.format(h4) + '{:x}'.format(h5) + '{:x}'.format(h6) + '{:x}'.format(h7)

    return digest

def rightrotate(value, shift):
    return (value >> shift) | (value << (64 - shift))

k = [
    0x428a2f98d728ae22, 0x7137449123ef65cd, 0xb5c0fbcfec4d3b2f, 0xe9b5dba58189dbbc,
    0x3956c25bf348b538, 0x59f111f1b605d019, 0x923f82a4af194f9b, 0xab1c5ed5da6d8118,
    0xd807aa98a3030242, 0x12835b0145706fbe, 0x243185be4ee4b28c, 0x550c7dc3d5ffb4e2,
    0x72be5d74f27b896f, 0x80deb1fe3b1696b1, 0x9bdc06a725c71235, 0xc19bf174cf692694,
    0xe49b69c19ef14ad2, 0xefbe4786384f25e3, 0x0fc19dc68b8cd5b5, 0x240ca1cc77ac9c65,
    0x2de92c6f592b0275, 0x4a7484aa6ea6e483, 0x5cb0a9dcbd41fbd4, 0x76f988da831153b5,
    0x983e5152ee66dfab, 0xa831c66d2db43210, 0xb00327c898fb213f, 0xbf597fc7beef0ee4,
    0xc6e00bf33da88fc2, 0xd5a79147930aa725, 0x06ca6351e003826f, 0x142929670a0e6e70,
    0x27b70a8546d22ffc, 0x2e1b21385c26c926, 0x4d2c6dfc5ac42a2e, 0x53380d139d95b3df,
    0x650a73548baf63de, 0x766a0abb3c77b2a8, 0x81c2c92e47edaee6, 0x92722c851482353b,
    0xa2bfe8a14cf10364, 0xa81a664bbc423001, 0xc24b8b70d0f89791, 0xc76c51a30654be30,
    0xd192e819d6ef5218, 0xd69906245565a910, 0xf40e35855771202a, 0x106aa07032bbd1b8,
    0x19a4c116b8d2d0c8, 0x1e376c085141ab53, 0x2748774cdf8eeb99, 0x34b0bcb5e19b48a8,
    0x391c0cb3c5c95a63, 0x4ed8aa4ae3418ac4, 0x5b9cca4f7763e373, 0x682e6ff3d6b2b8a3,
    0x748f82ee5defb2fc, 0x78a5636f43172f60, 0x84c87814a1f0ab72, 0x8cc702081a6439ec,
    0x90befffa23631e28, 0xa4506cebde82bde9, 0xbef9a3f7b2c67915, 0xc67178f2e372532b,
    0xca273eceea26619c, 0xd186b8c721c0c207, 0xeada7dd6cde0eb1e, 0xf57d4f7fee6ed178,
    0x06f067aa72176fba, 0x0a637dc5a2c898a6, 0x113f9804bef90dae, 0x1b710b35131c471b,
    0x28db77f523047d84, 0x32caab7b40c72493, 0x3c9ebe0a15c9bebc, 0x431d67c49c100d4c,
    0x4cc5d4becb3e42b6, 0x597f299cfc657e2a, 0x5fcb6fab3ad6faec, 0x6c44198c4a475817,
]
def recheck_password_strength(entropy):
    if 0 < entropy <= 35:
        print("Password Status (Based on Entropy): Very weak")
    elif 36 <= entropy <= 59:
        print("Password Status (Based on Entropy): Weak")
    elif 60 <= entropy <= 119:
        print("Password Status (Based on Entropy): Strong")
    elif entropy >= 120:
        print("Password Status (Based on Entropy): Very strong")
    else:
        print("Password Status (Based on Entropy): Retype the password")

def clear_screen():
    print("*", end="", flush=True)


def check_password_strength():
    while True:

        print("***********")
        print("PASSWORD STRENGTH CHECKER")
        print("***********\n")

        common = False  # Initialize common as False

        # Load common passwords from a file
        common_passwords = set()
        with open('common passwords.txt', 'r') as file:
            common_passwords.update(line.strip() for line in file)

        password = input("Enter a password: ")

        if is_common_password(password, common_passwords):
            print("This is a common password. Please choose a stronger one.")
        else:
            main(password)

        op = input("Do you want to generate a password again (y/n): ")
        if op.lower() != 'y':
            return  # Use 'return' to exit the function

if __name__ == '__main__':
    check_password_strength()

***********
PASSWORD STRENGTH CHECKER
***********

Enter a password: 123456
This is a common password. Please choose a stronger one.
Do you want to generate a password again (y/n): y
***********
PASSWORD STRENGTH CHECKER
***********

Enter a password: gchvhcvjm,.
*Salt: E-<m%zg'<vT7{B'k
Hashed Password: 14c2cb246ecc2d13709e8d5696b9e1019c781dd12570edaf9b2edaac9ade271357e422e82961a49a70bba6ce401993d82728ca33df9f64644e3955f43d45b1b
Checking Password Strength...███

Password Status: Strong
The entropy of this password is 61.10047736845401
Password Status (Based on Entropy): Strong
Do you want to generate a password again (y/n): n
